<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/crawer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os,re
from bs4 import BeautifulSoup
# 品書閣
url="https://tw.pinsuge.com/index/85767.html" #@param {type:'string'}
sites="https://tw.pinsuge.com" #@param {type:'string'}
req = requests.get(url)
stop_word="請大家收藏" #@param {type:'string'}
next_word="點下一章繼續"
soup=BeautifulSoup(req.text)

soup

In [ ]:
articles=soup.find_all(name="a")
links=[]

for itm in articles[1:]:
  if not itm.get("title"):
    continue 
  links.append([itm.get("title",),"{}{}".format(sites,itm.get("href"))])
try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
links

In [ ]:

def get_content(content):

  to_text=''
  for itm in content[:-1]:
    txt=itm.getText()
    if "一秒記住" in txt:
      continue
    if stop_word in txt:
      break
    
    to_text+="%s \n" %txt
  return to_text
  
  

index=1
for link in links:
  reg=requests.get(link[1])
  soup=BeautifulSoup(reg.text)
  soup.find(name="div",id="content").find_all(name="p")
  
  to_text=''
  to_text+="%s \n" %link[0]
  print(link[0])
  to_text+=get_content(soup.find(name="div",id="content").find_all(name="p"))
  if len(soup.find_all(name="p",string=re.compile(next_word)))>0:
    reg2=requests.get("{}{}".format(sites,soup.find("a",text="下一章").get("href")))
    to_text+=get_content(BeautifulSoup(reg2.text).find(name="div",id="content").find_all(name="p"))
  # print(to_text)

  with open("%03d.txt" % index,mode="w",encoding="utf-8") as f:
    f.write(to_text)
  index+=1


In [ ]:
files=os.listdir()
files=[file for file in files if file.endswith(".txt")]
files.sort()
files
for file in files:
  os.system("cat {}>> ../ouptput.txt".format(file))

from google.colab import files
files.download('../ouptput.txt') 
os.system("rm -fr *")